Autor: Werikson Alves

Codigo para calulo para fluxo de potencias pelo metodo de Newton-Raphson

In [1]:
import numpy as np
from sympy import * #var, Lambda, exp, log, sin, cos, tan, sqrt, ln

Parametros relacionados com a impedancia

In [2]:
# Dados iniciais [n,    Tipo,   V, teta,     P,     Q]
Matriz_Barras = [[1, 'VTeta',   1,    0,   '?',   '?'],
                 [2,    'PV',   1,  '?',   0.2,   '?'],
                 [3,    'PQ', '?',  '?', -0.15,  0.05]]

# Dados iniciais      [k, m, Z_km]
Matriz_Impedancias = [[1, 2, complex(0.015, 0.3)],
                      [2, 3, complex(0.015, 0.8)]]

y11 =  (Matriz_Impedancias[0][2])**(-1)
y12 = -(Matriz_Impedancias[0][2])**(-1)
y13 =  0
y21 =  y12
y22 =  (Matriz_Impedancias[0][2])**(-1) + (Matriz_Impedancias[1][2])**(-1)
y23 = -(Matriz_Impedancias[1][2])**(-1)
y31 =  y13
y32 =  y23
y33 =  (Matriz_Impedancias[1][2])**(-1)

Y = np.array([[y11, y12,y13], [y21, y22, y23], [y31, y32, y33]]) # Matriz Adimitancia

G = Y.real
B = Y.imag

print('\nY = ', Y)
print('\nG = ', G)
print('\nB = ', B)


Y =  [[ 0.16625104-3.32502078j -0.16625104+3.32502078j  0.        +0.j        ]
 [-0.16625104+3.32502078j  0.1896803 -4.57458148j -0.02342926+1.2495607j ]
 [ 0.        +0.j         -0.02342926+1.2495607j   0.02342926-1.2495607j ]]

G =  [[ 0.16625104 -0.16625104  0.        ]
 [-0.16625104  0.1896803  -0.02342926]
 [ 0.         -0.02342926  0.02342926]]

B =  [[-3.32502078  3.32502078  0.        ]
 [ 3.32502078 -4.57458148  1.2495607 ]
 [ 0.          1.2495607  -1.2495607 ]]


Parametros relacionados com as potencias

In [3]:
teta2,teta3,V3 = var('teta2 teta3 V3') # Variaveis simbolicas
teta1, V1, V2 = Matriz_Barras[0][3], Matriz_Barras[0][2], Matriz_Barras[1][2] # Variaveis numericas

# Potências esperadas nas barras 
P_esp_2 = Matriz_Barras[1][4]
P_esp_3 = Matriz_Barras[2][4]
Q_esp_3 = Matriz_Barras[2][5]

# Vetor de potencias
DeltaP2 = Lambda((teta2,teta3,V3), P_esp_2 - V2*( V1*(G[1,0]*cos(teta2-teta1) + B[1,0]*sin(teta2-teta1)) +
                                                  V3*(G[1,2]*cos(teta2-teta3) + B[1,2]*sin(teta2-teta3)) + 
                                                  V2*(G[1,1]*cos(teta2-teta2) + B[1,1]*sin(teta2-teta2)) ))

DeltaP3 = Lambda((teta2,teta3,V3), P_esp_3 - V3*( V3*(G[2,2]*cos(teta3-teta3) + B[2,2]*sin(teta3-teta3)) + 
                                             V2*(G[2,1]*cos(teta3-teta2) + B[2,1]*sin(teta3-teta2)) ))

DeltaQ3 = Lambda((teta2,teta3,V3), Q_esp_3 - V3*( V3*(G[2,2]*sin(teta3-teta3) - B[2,2]*cos(teta3-teta3)) + 
                                             V2*(G[2,1]*sin(teta3-teta2) - B[2,1]*cos(teta3-teta2)) ))

Delta_P_Q = [[DeltaP2],[DeltaP3],[DeltaQ3]]

for i in range(len(Delta_P_Q)):
    print('\nDelta_P_Q[',i+1,'] = ', Delta_P_Q[i][0](teta2,teta3,V3))


Delta_P_Q[ 1 ] =  -V3*(1.24956070131594*sin(teta2 - teta3) - 0.0234292631496739*cos(teta2 - teta3)) - 3.32502078137988*sin(teta2) + 0.166251039068994*cos(teta2) + 0.0103196977813318

Delta_P_Q[ 2 ] =  -V3*(0.0234292631496739*V3 - 1.24956070131594*sin(teta2 - teta3) - 0.0234292631496739*cos(teta2 - teta3)) - 0.15

Delta_P_Q[ 3 ] =  -V3*(1.24956070131594*V3 + 0.0234292631496739*sin(teta2 - teta3) - 1.24956070131594*cos(teta2 - teta3)) + 0.05


Parametros relacionados com a Jacobiana

In [4]:
# Matriz Jacobiana
j11 = Lambda((teta2,teta3,V3), diff(DeltaP2(teta2,teta3,V3),teta2))
j12 = Lambda((teta2,teta3,V3), diff(DeltaP2(teta2,teta3,V3),teta3))
j13 = Lambda((teta2,teta3,V3), diff(DeltaP2(teta2,teta3,V3),V3))

j21 = Lambda((teta2,teta3,V3), diff(DeltaP3(teta2,teta3,V3),teta2))
j22 = Lambda((teta2,teta3,V3), diff(DeltaP3(teta2,teta3,V3),teta3))
j23 = Lambda((teta2,teta3,V3), diff(DeltaP3(teta2,teta3,V3),V3))

j31 = Lambda((teta2,teta3,V3), diff(DeltaQ3(teta2,teta3,V3),teta2))
j32 = Lambda((teta2,teta3,V3), diff(DeltaQ3(teta2,teta3,V3),teta3))
j33 = Lambda((teta2,teta3,V3), diff(DeltaQ3(teta2,teta3,V3),V3))

J = [[j11, j12, j13],
     [j21, j22, j23],
     [j31, j32, j33]]

#J_inv = Inverse(J)

for i in range(len(J)):
    for j in range(len(J)):
        print('J[',i+1,'][',j+1,'] = ', J[i][j](teta2,teta3,V3))
    print('\n')


J[ 1 ][ 1 ] =  -V3*(0.0234292631496739*sin(teta2 - teta3) + 1.24956070131594*cos(teta2 - teta3)) - 0.166251039068994*sin(teta2) - 3.32502078137988*cos(teta2)
J[ 1 ][ 2 ] =  -V3*(-0.0234292631496739*sin(teta2 - teta3) - 1.24956070131594*cos(teta2 - teta3))
J[ 1 ][ 3 ] =  -1.24956070131594*sin(teta2 - teta3) + 0.0234292631496739*cos(teta2 - teta3)


J[ 2 ][ 1 ] =  -V3*(0.0234292631496739*sin(teta2 - teta3) - 1.24956070131594*cos(teta2 - teta3))
J[ 2 ][ 2 ] =  -V3*(-0.0234292631496739*sin(teta2 - teta3) + 1.24956070131594*cos(teta2 - teta3))
J[ 2 ][ 3 ] =  -0.0468585262993479*V3 + 1.24956070131594*sin(teta2 - teta3) + 0.0234292631496739*cos(teta2 - teta3)


J[ 3 ][ 1 ] =  -V3*(1.24956070131594*sin(teta2 - teta3) + 0.0234292631496739*cos(teta2 - teta3))
J[ 3 ][ 2 ] =  -V3*(-1.24956070131594*sin(teta2 - teta3) - 0.0234292631496739*cos(teta2 - teta3))
J[ 3 ][ 3 ] =  -2.49912140263189*V3 - 0.0234292631496739*sin(teta2 - teta3) + 1.24956070131594*cos(teta2 - teta3)




In [5]:
# Vetor inicial

teta2_inicial, teta3_inicial, V3_inicial = 0, 0, 1 # Chute inicial
teta_V = [[teta2_inicial], 
          [teta3_inicial],
          [V3_inicial]]

Delta_teta_V = [[0], 
                [0], 
                [0]]

# Erro
Ep = 0.001
Eq = 0.001

# Numeros de iterações
Iteracao = 0
Max_Iteracao = 10

J1 = [[0,0,0],[0,0,0],[0,0,0]]
for i in range(len(J)):
    for j in range(len(J)):
        J1[i][j] = np.float(J[i][j](teta_V[0][0],teta_V[1][0],teta_V[2][0]))
J_inv = np.linalg.inv(J1)


<ipython-input-5-493784f5b3d7>:23: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  J1[i][j] = np.float(J[i][j](teta_V[0][0],teta_V[1][0],teta_V[2][0]))


In [6]:
def Imprime(Iteracao, teta_V, Delta_P_Q, J, J_inv, Delta_teta_V):
    # Estado inicial
    print('Iteração: ', Iteracao)
    for i in range(len(teta_V)):
        print('teta_V[',i+1,'] = ', teta_V[i][0])
    print('\n')
    for i in range(len(Delta_P_Q)):
        print('Delta_P_Q[',i+1,'] = ', Delta_P_Q[i][0](teta_V[0][0],teta_V[1][0],teta_V[2][0]))
    print('\n')
    for i in range(len(J)):
        for j in range(len(J)):
            print('J[',i+1,'][',j+1,'] = ', J[i][j](teta_V[0][0],teta_V[1][0],teta_V[2][0]))
    print('\n')
    for i in range(len(J_inv)):
        for j in range(len(J_inv)):
            print('J_inv[',i+1,'][',j+1,'] = ', J_inv[i][j])
        print('\n')
    print('\n')
    for i in range(len(Delta_teta_V)):
        print('Delta_teta_V[',i+1,'] = ', Delta_teta_V[i][0])
    print('\n')


In [7]:
# Algoritmo de Newton-Raphson
Imprime(Iteracao, teta_V, Delta_P_Q, J, J_inv, Delta_teta_V)
while (Iteracao < Max_Iteracao):
    Iteracao += 1
    for i in range(len(Delta_teta_V)):
        Delta_teta_V[i][0] = 0
    for i in range(len(Delta_teta_V)):
        for j in range(len(Delta_P_Q)):
            Delta_teta_V[i][0] += J_inv[i][j]*Delta_P_Q[j][0](teta_V[0][0],teta_V[1][0],teta_V[2][0])
    for i in range(len(Delta_teta_V)):
        teta_V[i][0] -= Delta_teta_V[i][0]
    if (abs(Delta_teta_V[0][0]) < Ep and abs(Delta_teta_V[1][0]) < Ep and abs(Delta_teta_V[2][0]) < Eq):
        break
    Imprime(Iteracao, teta_V, Delta_P_Q, J,J_inv, Delta_teta_V)
    for i in range(len(J)):
        for j in range(len(J)):
            J1[i][j] = np.float(J[i][j](teta_V[0][0],teta_V[1][0],teta_V[2][0]))
    J_inv = np.linalg.inv(J1)

Iteração:  0
teta_V[ 1 ] =  0
teta_V[ 2 ] =  0
teta_V[ 3 ] =  1


Delta_P_Q[ 1 ] =  0.200000000000000
Delta_P_Q[ 2 ] =  -0.150000000000000
Delta_P_Q[ 3 ] =  0.0500000000000000


J[ 1 ][ 1 ] =  -4.57458148269583
J[ 1 ][ 2 ] =  1.24956070131594
J[ 1 ][ 3 ] =  0.0234292631496739
J[ 2 ][ 1 ] =  1.24956070131594
J[ 2 ][ 2 ] =  -1.24956070131594
J[ 2 ][ 3 ] =  -0.0234292631496739
J[ 3 ][ 1 ] =  -0.0234292631496739
J[ 3 ][ 2 ] =  0.0234292631496739
J[ 3 ][ 3 ] =  -1.24956070131594


J_inv[ 1 ][ 1 ] =  -0.30074999999999996
J_inv[ 1 ][ 2 ] =  -0.3007499999999999
J_inv[ 1 ][ 3 ] =  3.525512960640671e-19


J_inv[ 2 ][ 1 ] =  -0.30075
J_inv[ 2 ][ 2 ] =  -1.10075
J_inv[ 2 ][ 3 ] =  0.015000000000000001


J_inv[ 3 ][ 1 ] =  6.322265265609794e-19
J_inv[ 3 ][ 2 ] =  -0.015
J_inv[ 3 ][ 3 ] =  -0.8000000000000002




Delta_teta_V[ 1 ] =  0
Delta_teta_V[ 2 ] =  0
Delta_teta_V[ 3 ] =  0


Iteração:  1
teta_V[ 1 ] =  0.0150375000000000
teta_V[ 2 ] =  -0.105712500000000
teta_V[ 3 ] =  1.03775000000000


Del

<ipython-input-7-df58b2d5e248>:17: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  J1[i][j] = np.float(J[i][j](teta_V[0][0],teta_V[1][0],teta_V[2][0]))


In [8]:
P1 = Lambda((teta2,teta3,V3), V1*(V2*(G[0,1]*cos(teta1 - teta2) + B[0,1]*sin(teta1-teta2)) + 
                                  V1*(G[0,0]*cos(teta1 - teta1) + B[0,0]*sin(teta1-teta1))))

Q1 = Lambda((teta2,teta3,V3), V1*(V2*(G[0,1]*sin(teta1 - teta2) - B[0,1]*cos(teta1-teta2)) +
                                  V1*(G[0,0]*sin(teta1 - teta1) - B[0,0]*cos(teta1-teta1))))

Q2 = Lambda((teta2,teta3,V3), V2*(V1*(G[1,0]*sin(teta2 - teta1) - B[1,0]*cos(teta2-teta1)) +
                                  V2*(G[1,1]*sin(teta2 - teta2) - B[1,1]*cos(teta2-teta2)) + 
                                  V3*(G[1,2]*sin(teta2 - teta3) - B[1,2]*cos(teta2-teta3)) ))

Resp = [[P1],[Q1],[Q2]]
print(teta_V)
for i in range(len(Resp)):
    print('\nDelta_P_Q[',i+1,'] = ', np.float(Resp[i][0](teta_V[0][0],teta_V[1][0],teta_V[2][0])))

[[0.0149261279700415], [-0.102604576163412], [1.02976023687610]]

Delta_P_Q[ 1 ] =  -0.049609323747161926

Delta_P_Q[ 2 ] =  0.002851774787279826

Delta_P_Q[ 3 ] =  -0.03325031647075671


<ipython-input-8-442439a613ca>:14: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  print('\nDelta_P_Q[',i+1,'] = ', np.float(Resp[i][0](teta_V[0][0],teta_V[1][0],teta_V[2][0])))
